Since its inception, the National Domestic Workers Alliance has collected survey data about domestic workers and low-propensity voters of color. With the creation of a data department (myself and a former organizer), expanded capacity allowed for this data to be used in predictive analytics. Two models emerged: one to measure the likelihood of a respondent being a domestic worker (as defined by participation in the care economy) and one to measure the likelihood of member engagement with the organizational primary purpose.



In the domestic worker identification project, a machine learning model was trained on a large dataset of voter, demographic, immigration and consumer data to predict which respondents were most likely to identify as domestic workers. This model was then fine-tuned using a smaller dataset of voter demographic and consumer data from a 2020 COVID survey to predict which respondents were most likely to be domestic workers.

The first step of the project was to preprocess the data by cleaning and normalizing the data, and then encoding categorical variables. The demographic and consumer data were used as features, and the target variable was whether or not the respondent replied “Yes” to the general survey question “Are you a domestic worker?”.

A machine learning model was then trained on the large dataset using gradient boosting algorithm. This model was then fine-tuned using the smaller dataset from the 2020 COVID survey. The fine-tuning process involved re-training the model using the smaller dataset while keeping the pre-trained weights from the initial model as a starting point. This allowed the model to quickly adapt to the new dataset while still leveraging the knowledge learned from the larger dataset.

The fine-tuned model was then evaluated on a hold-out test set from the 2020 COVID survey and was found to have improved performance compared to a model trained from scratch on the smaller dataset.

This project demonstrates the effectiveness of transfer learning for tabular data, as the domestic worker model was able to leverage the knowledge learned from the larger dataset to improve its performance on the smaller dataset from 2020.